## Carefull, very much work in progress
Currently, uses a single items
This may not work
Other ideas:
Using multiple items

Uses scaler, may not be working correctly.

In [2]:
%load_ext dotenv
%reload_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
import mlflow
from darts.models import XGBModel
import pandas as pd
import numpy as np

In [5]:
# Download source data from s3
from ml.common.folder_s3_sync import download_folder_from_s3

datapath = "../../../data/train_xgboost/"


download_folder_from_s3(s3_folder_prefix='datasources/gw2/raw/1762686861', local_folder=datapath)
df = pd.read_csv(f'{datapath}Copper Ore.csv', delimiter=";")


Source: ost-s3/datasources/gw2/raw/1762686861
Target: ../../../data/train_xgboost/

Found 60 files to download

✓ +1 Agony Infusion.csv
✓ Auric Sliver.csv
✓ Bag of Coffee Beans.csv
✓ Bag of Sugar.csv
✓ Black Lion Chest.csv
✓ Bowl of Candy Corn Custard.csv
✓ Choya Spine.csv
✓ Congealed Putrescence.csv
✓ Copper Ore.csv
✓ Cup of Spiced Apple Cider.csv
✓ Elder Wood Log.csv
✓ Eye of Kormir.csv
✓ Flawless Snowflake.csv
✓ Fried Golden Dumpling.csv
✓ Glass of Buttered Spirits.csv
✓ Glob of Ectoplasm.csv
✓ Gold Ingot.csv
✓ Gold Ore.csv
✓ Gossamer Scrap.csv
✓ Green Wood Log.csv
✓ Handful of Red Lentils.csv
✓ Iron Ore.csv
✓ Jug of Water.csv
✓ Jute Scrap.csv
✓ Large Bone.csv
✓ Large Claw.csv
✓ Large Fang.csv
✓ Large Scale.csv
✓ Ley-Infused Sand.csv
✓ Lucent Mote.csv
✓ Lucky Draketail.csv
✓ Lucky Prismatic Rocket.csv
✓ Lump of Coal.csv
✓ Mithril Ore.csv
✓ Mystic Coin.csv
✓ Orichalcum Ore.csv
✓ Piece of Candy Corn.csv
✓ Piece of Common Unidentified Gear.csv
✓ Piece of Unidentified Gear.csv
✓ Piece o

In [ ]:
from darts import TimeSeries
#value_cols = ["buy_unit_price","sell_quantity", "sell_unit_price", "buy_quantity"]

value_cols = ["sell_unit_price"]

tsdf = df[value_cols + ['fetched_at']].copy()
# series = TimeSeries.from_dataframe(df, time_col="fetched_at")

# Convert to datetime and set as index
tsdf['fetched_at'] = pd.to_datetime(tsdf['fetched_at'])
tsdf = tsdf.set_index('fetched_at')

#value_cols = ["buy_unit_price"]  #ARIMA only univariate !

# Resample to exact 5-minute intervals, forward-filling missing values
tsdf_resampled = tsdf[value_cols].resample('5min').mean().interpolate(method='linear')

# Create TimeSeries (no need for fill_missing_dates now)
series = TimeSeries.from_dataframe(tsdf_resampled, value_cols=value_cols)
series.plot();


In [ ]:
# Scaling the data
from darts.dataprocessing.transformers import Scaler
scaler = Scaler()
series_scaled = scaler.fit_transform(series)
series_scaled.plot()

In [ ]:
train, val = series_scaled.split_after(0.75)
#train, val = series.split_after(0.75)
train.plot(label="training")
val.plot(label="validation");


In [ ]:
from darts.metrics import mape
from darts.models import XGBModel

model = XGBModel(
    lags=48,                     # use the last 12 observations as features
    lags_past_covariates=None,   # no past covariates by default
    lags_future_covariates=None, # no future covariates by default
    output_chunk_length=1,       # one‑step‑ahead forecasts
    n_estimators=200,            # number of boosting rounds
    max_depth=40,                 # tree depth
    learning_rate=0.1,           # shrinkage factor
    subsample=0.8,               # row sampling ratio
    colsample_bytree=0.8,        # column sampling ratio
    reg_alpha=0.0,               # L1 regularisation
    reg_lambda=0.0,              # L2 regularisation
    #early_stopping_rounds=20,   # stop if validation error doesn’t improve
    device="cuda:0" # this is broken
)
model.fit(train, val_series=val)
forecast = model.predict(n=len(val), series=train)
error = mape(val, forecast)
print(f"MAPE: {error:.2f}%")


In [ ]:
forecast_reversed = scaler.inverse_transform(forecast)
val_reversed = scaler.inverse_transform(val)
forecast_reversed.plot()
val_reversed.plot()

In [9]:
from darts.metrics import mape
from darts.models import XGBModel
from tqdm.notebook import tqdm
import mlflow
import os
from src.utils.mlflow_helper import setup_mlflow

setup_mlflow('gw2_xgboost', 'gw2_xgboost')
print(mlflow.get_tracking_uri())
os.environ["MLFLOW_S3_ENDPOINT_URL"] = "http://localhost:9000"
os.environ['AWS_ACCESS_KEY_ID'] = 'minio'
os.environ['AWS_SECRET_ACCESS_KEY'] = 'minio123'

best_mape = float('inf')
best_params = None

# Define hyperparameter ranges
lags_range = [24, 48, 72]
n_estimators_range = [100, 200, 300]
max_depth_range = [20, 40, 60]
learning_rate_range = [0.05, 0.1, 0.2]
subsample_range = [0.6, 0.8, 1.0]
colsample_bytree_range = [0.6, 0.8, 1.0]

# Calculate total combinations
total_combinations = len(lags_range) * len(n_estimators_range) * len(max_depth_range) * \
                    len(learning_rate_range) * len(subsample_range) * len(colsample_bytree_range)

# Parent run for the hyperparameter search
with mlflow.start_run(run_name="XGBOOST_Hyperparameter_Search"):

    # Log search configuration
    mlflow.log_param("search_lags", str(lags_range))
    mlflow.log_param("search_n_estimators", str(n_estimators_range))
    mlflow.log_param("search_max_depth", str(max_depth_range))
    mlflow.log_param("search_learning_rate", str(learning_rate_range))
    mlflow.log_param("search_subsample", str(subsample_range))
    mlflow.log_param("search_colsample_bytree", str(colsample_bytree_range))
    mlflow.log_param("total_combinations", total_combinations)

    # Create progress bar
    combination_count = 0
    for lags in lags_range:
        for n_estimators in n_estimators_range:
            for max_depth in max_depth_range:
                for learning_rate in learning_rate_range:
                    for subsample in subsample_range:
                        for colsample_bytree in colsample_bytree_range:
                            combination_count += 1

                            print(f"Testing XGBModel {combination_count}/{total_combinations}: "
                                  f"lags={lags}, n_est={n_estimators}, depth={max_depth}, "
                                  f"lr={learning_rate}, sub={subsample}, col={colsample_bytree}")

                            # Child run for each parameter combination
                            with mlflow.start_run(run_name=f"XGB_{lags}_{n_estimators}_{max_depth}_{learning_rate}_{subsample}_{colsample_bytree}", nested=True):
                                # Log parameters
                                mlflow.log_param("lags", lags)
                                mlflow.log_param("n_estimators", n_estimators)
                                mlflow.log_param("max_depth", max_depth)
                                mlflow.log_param("learning_rate", learning_rate)
                                mlflow.log_param("subsample", subsample)
                                mlflow.log_param("colsample_bytree", colsample_bytree)
                                mlflow.log_param("output_chunk_length", 1)
                                mlflow.log_param("reg_alpha", 0.0)
                                mlflow.log_param("reg_lambda", 0.0)
                                mlflow.log_param("model_type", "XGBModel")

                                try:
                                    # Train model
                                    model = XGBModel(
                                        lags=lags,
                                        lags_past_covariates=None,
                                        lags_future_covariates=None,
                                        output_chunk_length=1,
                                        n_estimators=n_estimators,
                                        max_depth=max_depth,
                                        learning_rate=learning_rate,
                                        subsample=subsample,
                                        colsample_bytree=colsample_bytree,
                                        reg_alpha=0.0,
                                        reg_lambda=0.0,
                                        # device="cuda:0"  # commented out as is broken
                                    )
                                    model.fit(train)

                                    # Make predictions
                                    forecast = model.predict(len(val))
                                    error = mape(val, forecast)

                                    # Log metrics
                                    mlflow.log_metric("mape", error)
                                    mlflow.log_metric("mape_percentage", error)

                                    # Log model artifact
                                    mlflow.sklearn.log_model(model, "model")

                                    # Check if this is the best model so far
                                    if error < best_mape:
                                        best_mape = error
                                        best_params = {
                                            'lags': lags,
                                            'n_estimators': n_estimators,
                                            'max_depth': max_depth,
                                            'learning_rate': learning_rate,
                                            'subsample': subsample,
                                            'colsample_bytree': colsample_bytree
                                        }
                                        print(f"New best: {best_params} with MAPE: {error:.2f}%")

                                        # Log as best model in parent run
                                        mlflow.log_metric("best_mape", error)
                                        mlflow.log_param("best_lags", lags)
                                        mlflow.log_param("best_n_estimators", n_estimators)
                                        mlflow.log_param("best_max_depth", max_depth)
                                        mlflow.log_param("best_learning_rate", learning_rate)
                                        mlflow.log_param("best_subsample", subsample)
                                        mlflow.log_param("best_colsample_bytree", colsample_bytree)

                                except Exception as e:
                                    # Log failed runs
                                    mlflow.log_param("status", "failed")
                                    mlflow.log_param("error", str(e))
                                    print(f"Failed XGBModel combination: {str(e)}")

    # Log final results in parent run
    mlflow.log_metric("final_best_mape", best_mape)
    mlflow.log_param("final_best_params", str(best_params))

    print(f"\nBest model: {best_params} with MAPE: {best_mape:.2f}%")


2025/11/11 00:59:03 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 00:59:08 WARNING mlflow.utils.requirements_utils: Found amdsmi version (26.1.0+5df6c765) contains a local version label (+5df6c765). MLflow logged a pip requirement for this package as 'amdsmi==26.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 00:59:08 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 00:59:08 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+rocm7.1.0.git824e8c87) contains a local ve

🏃 View run XGB_48_200_40_0.05_0.8_1.0 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4/runs/b14a5689a2e344f09223360f30305cdd
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4
Testing XGBModel 358/729: lags=48, n_est=200, depth=40, lr=0.05, sub=1.0, col=0.6


2025/11/11 00:59:21 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 00:59:27 WARNING mlflow.utils.requirements_utils: Found amdsmi version (26.1.0+5df6c765) contains a local version label (+5df6c765). MLflow logged a pip requirement for this package as 'amdsmi==26.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 00:59:27 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 00:59:27 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+rocm7.1.0.git824e8c87) contains a local ve

🏃 View run XGB_48_200_40_0.05_1.0_0.6 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4/runs/570d59ac2a1a47569061ba1637d4b113
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4
Testing XGBModel 359/729: lags=48, n_est=200, depth=40, lr=0.05, sub=1.0, col=0.8


2025/11/11 00:59:39 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 00:59:44 WARNING mlflow.utils.requirements_utils: Found amdsmi version (26.1.0+5df6c765) contains a local version label (+5df6c765). MLflow logged a pip requirement for this package as 'amdsmi==26.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 00:59:44 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 00:59:44 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+rocm7.1.0.git824e8c87) contains a local ve

...

🏃 View run XGB_72_300_60_0.2_1.0_0.6 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4/runs/2683b71e50e34bc2806576b990083e85
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4
Testing XGBModel 728/729: lags=72, n_est=300, depth=60, lr=0.2, sub=1.0, col=0.8


2025/11/11 04:13:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 04:14:07 WARNING mlflow.utils.requirements_utils: Found amdsmi version (26.1.0+5df6c765) contains a local version label (+5df6c765). MLflow logged a pip requirement for this package as 'amdsmi==26.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 04:14:07 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 04:14:07 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+rocm7.1.0.git824e8c87) contains a local ve

🏃 View run XGB_72_300_60_0.2_1.0_0.8 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4/runs/07cd7cd84da24a98ac83bd3b2ecae52b
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4
Testing XGBModel 729/729: lags=72, n_est=300, depth=60, lr=0.2, sub=1.0, col=1.0


2025/11/11 04:14:24 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/11 04:14:33 WARNING mlflow.utils.requirements_utils: Found amdsmi version (26.1.0+5df6c765) contains a local version label (+5df6c765). MLflow logged a pip requirement for this package as 'amdsmi==26.1.0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 04:14:33 WARNING mlflow.utils.requirements_utils: Found lz4 version (4.0.2+dfsg) contains a local version label (+dfsg). MLflow logged a pip requirement for this package as 'lz4==4.0.2' without the local version label to make it installable from PyPI. To specify pip requirements containing local version labels, please use `conda_env` or `pip_requirements`.
2025/11/11 04:14:33 WARNING mlflow.utils.requirements_utils: Found torchvision version (0.23.0+rocm7.1.0.git824e8c87) contains a local ve

🏃 View run XGB_72_300_60_0.2_1.0_1.0 at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4/runs/515abf8b348c4d3993f00789ed2976b0
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4

Best model: {'lags': 72, 'n_estimators': 300, 'max_depth': 60, 'learning_rate': 0.2, 'subsample': 0.6, 'colsample_bytree': 0.8} with MAPE: 22.66%
🏃 View run XGBOOST_Hyperparameter_Search at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4/runs/735b223fc7274a03af6690b5d2bb349c
🧪 View experiment at: http://lukas-hp-z2-mini-g1a-workstation-desktop-pc.tail7adb49.ts.net:5000/#/experiments/4


In [10]:
from darts.models import NaiveDrift, VARIMA
from darts.metrics import mape

def eval_model(model):

    model.fit(train)
    forecast = model.predict(len(val))
    print(f"model {model} obtains MAPE: {mape(val, forecast):.2f}%")


eval_model(NaiveDrift())
eval_model(VARIMA())
eval_model(VARIMA(p=3, d=1, q=2))

ValueError: Model `VARIMA` only supports multivariate TimeSeries instances


model NaiveDrift() obtains MAPE: 50.30%


ValueError: Model `VARIMA` only supports multivariate TimeSeries instances